In [ ]:
import configobj
from crate import client
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("nbagg")
import matplotlib.pyplot as plt
from teams import nba_teams
team_keys = nba_teams

In [ ]:
dfsub = pd.read_excel("../nba_2016_2017.xlsx")

print dfsub

dfsub_dict = dfsub.to_dict('list')
home_team = []
away_team = []

for full_team, vs_team in zip(dfsub_dict['Team'],dfsub_dict['Vs']):
    
    if vs_team.split()[0]=='@':
        
        guesses = process.extract(full_team, nba_teams.keys(), limit=4)
        matches = []
        for guess in guesses:
            ratio = fuzz.ratio(guess, full_team)
            matches.append({'team': guess[0],
                            'fuzz': guess[1],
                            'ratio': ratio})
        max_match = max(matches, key=lambda x: x['ratio'])
        team_longname = max_match['team']        
        #print full_team, team_longname, team_keys[team_longname]
        away_team.append(team_keys[team_longname])
        
        guesses = process.extract(vs_team, nba_teams.keys(), limit=4)
        matches = []
        for guess in guesses:
            ratio = fuzz.ratio(guess, vs_team)
            matches.append({'team': guess[0],
                            'fuzz': guess[1],
                            'ratio': ratio})
        max_match = max(matches, key=lambda x: x['ratio'])
        team_longname = max_match['team']
        #print vs_team, team_longname, team_keys[team_longname]
        home_team.append(team_keys[team_longname])
        
    else:

        guesses = process.extract(vs_team, nba_teams.keys(), limit=4)
        matches = []
        for guess in guesses:
            ratio = fuzz.ratio(guess, vs_team)
            matches.append({'team': guess[0],
                            'fuzz': guess[1],
                            'ratio': ratio})

        max_match = max(matches, key=lambda x: x['ratio'])
        team_longname = max_match['team']
        #print vs_team, team_longname, team_keys[team_longname]
        away_team.append(team_keys[team_longname])
        
        guesses = process.extract(full_team, nba_teams.keys(), limit=4)
        matches = []
        for guess in guesses:
            ratio = fuzz.ratio(guess, full_team)
            matches.append({'team': guess[0],
                            'fuzz': guess[1],
                            'ratio': ratio})
        max_match = max(matches, key=lambda x: x['ratio'])
        team_longname = max_match['team']
        #print full_team, team_longname, team_keys[team_longname]
        home_team.append(team_keys[team_longname])
       
dfsub_dict["Home"]=home_team
dfsub_dict["Visitor"]=away_team

dfsub = pd.DataFrame(dfsub_dict)

print dfsub.head(100)

In [ ]:
games = []

nrows = len(dfsub.Season)

for i,row in enumerate(dfsub.iterrows()):
    if i%10.==0:
        print i, nrows
    for row2 in dfsub.iterrows():
        if row[1]['Date']==row2[1]['Date'] and row[1]['Pts Scr']==row2[1]['Pts Alw.1'] and row[1]['O/U Line']==row2[1]['O/U Line'] and row[1]['Type']==row2[1]['Type']:
            
            if row[1]['FIELD']=='HOME':
                home_score = row[1]['Pts Scr']
                away_score = row[1]['Pts Alw.1']
                home_line = row[1]['Side Line']
                away_line = row2[1]['Side Line']
                over_under = row[1]['O/U Line']
                gametype = row[1]['Type']
            else:
                home_score = row[1]['Pts Alw.1']
                away_score = row[1]['Pts Scr']
                home_line = row2[1]['Side Line']
                away_line = row[1]['Side Line']
                over_under = row[1]['O/U Line']
                gametype = row[1]['Type']
            
            date = row[1]['Date']
            intdate = int(date.strftime('%Y%m%d'))
            
            games.append({'Home':row[1]['Home'],
                          'Away':row[1]['Visitor'],
                          'Date':intdate,
                          'Home Score':home_score,
                          'Away Score':away_score,
                          'Home Line':home_line,
                          'Away Line':away_line,
                          'Over Under':over_under,
                          'Game Type':gametype})

In [ ]:
dfgames = pd.DataFrame(games)
dfgames.drop_duplicates(inplace='True')
print dfgames

In [ ]:
dictgames = dfgames.to_dict('records')
print len(dictgames)
print dictgames[0]

In [ ]:
config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

for game in dictgames:
    
        game_id = str(game['Date']) + '_' + game['Away'] + '_' + game['Home']
        #check if the game is already in the databse
        query = '''SELECT * from nba.games WHERE "GameId"='%s' '''%game_id

        cursor.execute(query)
        if len([row for row in cursor]) == 0:

            #games
            row = [game_id, game['Home'], game['Away'], game['Date']]
            query = '''INSERT INTO nba.games ("GameId","HomeTeam", "AwayTeam", "GameDate") VALUES (?,?,?,?)'''
            cursor.execute(query, row)
            #lines
            row = [game_id, game['Home Line'], game['Over Under']]
            query = '''INSERT INTO nba.lines ("GameId", "HomeSpread", "OverUnder")
                    VALUES (?,?,?)'''
            cursor.execute(query, row)
            #results
            row = [game_id, game['Home Score'], game['Away Score']]
            query = '''INSERT INTO nba.results ("GameId", "HomeScore", "AwayScore")
                    VALUES (?,?,?)'''
            cursor.execute(query, row)